# Load files from data

**load all the articles from local storage (new_data & data) and prepare the tables for the subsequent analysis. ALternatively - if implemented - use boto3 to load data from S3 storage on AWS:**

In [ ]:
#client = boto3.client('s3')
#s3_resource = boto3.resource('s3')
#client.list_buckets()

---

### Loading packages

In [1]:
import pandas as pd
import numpy as np
import glob
import re
import string
#import boto3

### Loading data

The for-loop gathers all the .csv files within the data directory, concats and labels them and then returns a data frame.

In [2]:
category = {
    1: 'politik', 2: 'wirtschaft', 3: 'finanzen', 4: 'feuilleton', 5: 'sport', 6: 'gesellschaft', 7: 'stil', 
    8: 'technik-motor', 9: 'wissen', 10: 'reise', 11: 'beruf-chance', 12: 'aktuell'
}

In [3]:
def build_tables(category):
    raw_articles = []
    if category is 'aktuell':
        path = f"../new_data/aktuell/"
    else:
        path = f"../new_data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_articles.append(df)
    faz_articles = pd.concat(raw_articles, axis=0, ignore_index=True)
    faz_articles['label'] = category

    return faz_articles

---
# Generate Test and Train Data

**This for-loop goes through the data frames built in the first step and then collects the words from each entry in a list.**

## Train Data:

In [4]:
frames = []

for key, value in category.items():
    raw_faz = build_tables(value)
    frames.append(raw_faz)
    faz_train = pd.concat(frames, axis=0, ignore_index=True)

In [5]:
faz_train = faz_train.drop_duplicates()

# Split train data in 'politik' and 'rest':

This step is for the sake of labelling the subsequant data and to get an idea of the data distribution (**'Politics' make up about 17% of the train data**):

In [6]:
faz_rest = faz_train[faz_train.label != 'politik']
faz_rest.count()

link         1185
published    1185
title        1185
detailed     1185
label        1185
dtype: int64

In [7]:
faz_pol = faz_train[faz_train.label == 'politik']
faz_pol.count()

link         209
published    209
title        209
detailed     209
label        209
dtype: int64

In [8]:
209/1185

0.17637130801687764

---

## Build feature word list:

In [9]:
rgx_singles = re.compile("([\w][\w']*[\w])")
rgx_doubles = re.compile("([\w][\w']*[\w] +[\w][\w']*[\w])")
rgx_triples = re.compile("([\w][\w']*[\w] +[\w][\w']*[\w] +[\w][\w']*[\w])")

translator = str.maketrans('', '', string.punctuation)

In [10]:
#words = []
#
#for index, row in faz_train.iterrows():
#    words += list(row['detailed'].split(' '))

In [11]:
words = []

for index, row in faz_train.iterrows():
    line = row['detailed'].translate(translator) 
    words += rgx_singles.findall(line)
    words += rgx_doubles.findall(line)
    words += rgx_triples.findall(line)

In [12]:
word_list = pd.DataFrame(words, columns=['word'])

In [13]:
titles = []

for index, row in faz_train.iterrows():
    line = row['title'].translate(translator) 
    titles += rgx_singles.findall(line)
    titles += rgx_doubles.findall(line)
    titles += rgx_triples.findall(line)

In [14]:
title_list = pd.DataFrame(titles, columns=['word'])

In [15]:
scraped = [word_list, title_list]
merged = pd.concat(scraped)

In [16]:
merged['count'] = 1

In [17]:
features = merged.groupby('word').count()

In [18]:
features = features.sort_values('count', ascending=False)
feature_list = features.head(300)
feature_list = list(feature_list.index)

In [19]:
dictionary={}

for i in range(len(feature_list)):
    dictionary[feature_list[i]] = i

# Iterate over 'Politik' buzzwords:

In [20]:
pol_buzzword = []

for index, row in faz_pol.iterrows():
    new_line = np.zeros(300)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    pol_buzzword.append(new_line)

In [21]:
df1 = pd.DataFrame(pol_buzzword, columns=feature_list)

In [22]:
df1['goal_val'] = 1

# Iterate over remaining buzzwords:

In [23]:
buzzwords = []

for index, row in faz_rest.iterrows():
    new_line = np.zeros(300)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    buzzwords.append(new_line)

In [24]:
df2 = pd.DataFrame(buzzwords, columns=feature_list)

In [25]:
df2['goal_val'] = 0

In [26]:
train_tables = [df1, df2]
top_train_data = pd.concat(train_tables)

---
**saving the train data frames to a .csv file:**

In [27]:
name = 'train_data'

path = "../data_frames/" + name
print("Pathname:", path)

Pathname: ../data_frames/train_data


In [28]:
top_train_data.to_csv(path, index=False)

# SAME FOR TEST DATA:

In [29]:
def build_test_tables(category):
    raw_article = []
    if category is 'aktuell':
        path = f"../data/aktuell/"
    else:
        path = f"../data/{category}/"
        
    all_files = glob.glob(path + '*.csv')

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        raw_article.append(df)
    faz_article = pd.concat(raw_article, axis=0, ignore_index=True)
    faz_article['label'] = category

    return faz_article

In [30]:
fram = []

for key, value in category.items():
    raw = build_test_tables(value)
    fram.append(raw)
    faz_test = pd.concat(fram, axis=0, ignore_index=True)

In [31]:
faz_test

,link,published,title,detailed,label
0,https://www.faz.net/aktuell/politik/trumps-pra...,"Tue, 02 Apr 2019 14:16:22 +0200",Joe Biden: Vom Hoffnungsträger zum Problem?,Für viele Amerikaner ein vertrauter Politiker:...,politik
1,https://www.faz.net/aktuell/brexit/nach-brexit...,"Tue, 02 Apr 2019 07:46:28 +0200",Brexit: Das Ende naht – tut es das?,Eine Brexit-Anhängerin vor dem Parlament in Lo...,politik
2,https://www.faz.net/aktuell/politik/ausland/eu...,"Tue, 02 Apr 2019 12:30:17 +0200",EU: Risiko eines Chaos-Brexits wächst täglich,Michel BarnierWieder hat das britische Unterha...,politik
3,https://www.faz.net/aktuell/politik/ausland/ve...,"Tue, 02 Apr 2019 03:57:03 +0200",Venezuelas Oppositionschef Guaidó soll die Imm...,Juan Guaido (hier am Montag in der Universität...,politik
4,https://www.faz.net/aktuell/politik/alle-alter...,"Tue, 02 Apr 2019 00:10:13 +0200",Alle Alternativen zu Mays Brexit-Vertrag schei...,Am britischen Parlament scheint eine Projektio...,politik
5,https://www.faz.net/aktuell/politik/ausland/na...,"Mon, 01 Apr 2019 21:11:30 +0200",Nach der Wahl in der Türkei: Risse im System E...,Flagge zeigen: Die Kommunalwahl ist ein wichti...,politik
6,https://www.faz.net/aktuell/politik/ausland/sc...,"Mon, 01 Apr 2019 16:38:02 +0200",Interview mit Schwedens Ministerpräsident,Der schwedische Ministerpräsident Stefan Löfve...,politik
7,https://www.faz.net/aktuell/politik/ausland/sp...,"Mon, 01 Apr 2019 19:51:45 +0200",„Angriff auf die Republik“: Sprengsätze kurz v...,Donnerstag reist er nach Korsika: Emmanuel Mac...,politik
8,https://www.faz.net/aktuell/politik/inland/meh...,"Mon, 01 Apr 2019 18:30:00 +0200",Mehrheit der Deutschen lehnt gendergerechte Sp...,Wort mit Sternchen: Wird gendergerechte Sprach...,politik
9,https://www.faz.net/aktuell/politik/ausland/su...,"Wed, 03 Apr 2019 11:16:50 +0200",Sultanat Brunei: Homosexuellen droht nun die T...,Der Sultan von Brunei Hassanal Bolkiah herrsch...,politik


In [32]:
faz_test = faz_test.drop_duplicates()

---

In [33]:
test_pol = faz_test[faz_test.label == 'politik']
test_pol.count()

link         40
published    40
title        40
detailed     40
label        40
dtype: int64

In [34]:
test_rest = faz_test[faz_test.label != 'politik']
test_rest.count()

link         348
published    348
title        348
detailed     348
label        348
dtype: int64

In [35]:
40/348

0.11494252873563218

In [36]:
test_pol_buzzword = []

for index, row in test_pol.iterrows():
    new_line = np.zeros(300)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    test_pol_buzzword.append(new_line)

In [37]:
df3 = pd.DataFrame(test_pol_buzzword, columns=feature_list)

In [38]:
df3['goal_val'] = 1

In [39]:
test_rest_buzzword = []

for index, row in test_rest.iterrows():
    new_line = np.zeros(300)
    words = list(row['detailed'].split(' '))
    for word in words:
        if word in dictionary:
            new_line[dictionary[word]] += 1
    test_rest_buzzword.append(new_line)

In [40]:
df4 = pd.DataFrame(test_rest_buzzword, columns=feature_list)

In [41]:
df4['goal_val'] = 0

In [42]:
test_tables = [df3, df4]
top_test_data = pd.concat(test_tables)

---
**saving the data frames to a .csv file:**

In [43]:
name = 'test_data'

path = "../data_frames/" + name
print("Pathname:", path)

Pathname: ../data_frames/test_data


In [44]:
top_test_data.to_csv(path, index=False)

---